<a href="https://colab.research.google.com/github/TabeaPreusser/Horse_Detection/blob/master/rider_detection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.83 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (569/569), done.


In [2]:
import os
os.chdir('./Mask_RCNN')
!ls
!python setup.py install
!pip show mask-rcnn

assets	LICENSE      mrcnn	requirements.txt  setup.cfg
images	MANIFEST.in  README.md	samples		  setup.py
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating b

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


New Version with seperate classes

Rider Class

In [5]:
import os
import pandas as pd
from os.path import join
import cv2
import moviepy.editor as mpe



from matplotlib import pyplot
from matplotlib.patches import Rectangle

from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN


from numpy import zeros
from numpy import asarray

import math
from numba import jit
from tqdm import tnrange

# class that defines and loads the dataset for train and test
class RiderDataset(Dataset):
   
    def getClassname(self,index):
        class_names = ['BG', 'horse', 'rider']
        if index>=0 and index<3 :
          return class_names[i]
        else:
          return -1
    def load_dataset(self, dataset_path, is_train=True):
        # define classes
        self.add_class("dataset", 1, "horse")
        self.add_class("dataset", 2, "rider")

        FRAME_SOURCE = "./Horse_Detection/rimondo_frames"
        # where to find the labels
        db = "./Horse_Detection/annotations"

        videos = [d for d in os.listdir(FRAME_SOURCE) if os.path.isdir(join(FRAME_SOURCE, d))]
        counter = 0
        for vid in videos:
            video_path = join(FRAME_SOURCE, vid)
            frames = [f for f in os.listdir(video_path) if f.endswith(".png")]
            numberDataset = len(frames)
            borderTest = round(numberDataset * 0.9)

            for frame in frames:
                counter = counter + 1

                image_path = FRAME_SOURCE + "/" + vid + "/" + frame
                image_id = vid + "/" + frame
                # skip all test images if we are building the train set
                if is_train and counter <= borderTest:
                    continue
                # skip all train images if we are building the test/val set
                if not is_train and counter > borderTest:
                    continue
                # add to dataset
                self.add_image('dataset', image_id=counter, path=image_path, annotation=db)

    def extract_boxes(self, img_path, db):
        fileName, fileExtension = os.path.splitext(img_path)
        fileName = join(fileName, '.jpg')
        # get image name for search in database
        path_list = fileName.split(os.sep)
        imageNameDatabase = path_list[-3] + '/' + path_list[-2] + path_list[-1]
        db_path = join(db, imageNameDatabase)
        db_path = os.path.splitext(db_path)[0] + '.csv'
        db_path = os.path.normpath(db_path)
        label_data = pd.read_csv(db_path)
        allEntries = label_data.loc[(label_data["image"] == imageNameDatabase)]

        img = cv2.imread(img_path)
        height = img.shape[0]
        width = img.shape[1]
        bbox = []
        for _, entry in allEntries.iterrows():

            x = entry["x"] * width
            y = entry["y"] * height
            w = entry["width"] * width
            h = entry["height"] * height
            # Eckpunkte bbox berechnen
            xmin = round(x - w // 2)
            xmax = round(x + w // 2)
            ymin = round(y - h // 2)
            ymax = round(y + h // 2)
            if (entry["label"] == 1 or entry["label"] == 2):
                if (math.isnan(entry["x"]) or math.isnan(entry["y"]) or math.isnan(entry["width"]) or math.isnan(
                        entry["height"])):
                    continue
                box = [entry["label"], xmin, ymin, xmax, ymax]
                bbox.append(box)
        return bbox, height, width

    # load the masks for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location
        db_path = info['annotation']
        img_path = info['path']

        # load boxes
        boxes, h, w = self.extract_boxes(img_path, db_path)
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[2], box[4]
            col_s, col_e = box[1], box[3]
            masks[row_s:row_e, col_s:col_e, i] = 1

            if (box[0] == 1):
                class_ids.append(self.class_names.index('horse'))
            if (box[0] == 2):
                class_ids.append(self.class_names.index('rider'))
        return masks, asarray(class_ids, dtype='int32')

    # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']




Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1400832/45929032 bytes (3.0%)5701632/45929032 bytes (12.4%)10149888/45929032 bytes (22.1%)14483456/45929032 bytes (31.5%)18735104/45929032 bytes (40.8%)23019520/45929032 bytes (50.1%)27312128/45929032 bytes (59.5%)31662080/45929032 bytes (68.9%)36085760/45929032 bytes (78.6%)40476672/45929032 bytes (88.1%)44908544/45929032 bytes (97.8%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

Using TensorFlow backend.


Configuration Class

In [0]:
# define the prediction configuration
class RiderConfig(Config):
    # define the name of the configuration
    NAME = "rider_cfg"
    # number of classes (background + rider+horse)
    NUM_CLASSES = 1 + 2
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    #
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 50

Detection Class

In [0]:
# Detection of rider by bounding boxes
class rider_detection:

  def __init__(self, path_weights):
    self.model=self.setModel(path_weights)

  def setModel(self,path_weights):
        config = RiderConfig()
        # define the model
        model = MaskRCNN(mode='inference', model_dir='./', config=config)
        # load trained weights 
        model.load_weights(path_weights, by_name=True)
        return model

  # get bounding boxes for one image
  def getBboxes(self,im):
	  results = self.model.detect([im], verbose=0)
	  r = results[0]
	  # extract ids and bboxes for riders
	  rois = r['rois']

	  bbox = []
	  counter=-1
	  for i in r['class_ids']:
		  counter=counter+1
		  b=self.box_to_dict(rois[counter])
		  bbox.append(b)
  	  # austauschen mit crop methode statt zeichnen
	    #drawBox(imagepath, bbox)
	  return bbox
  
  # get bounding boxes for list of frames
  def getBboxes_framelist(self,frames):
    bbox_list=list()
    for i in range(len(frames)):
      bbox_list.append(self.getBboxes(frames[i]))
    return bbox_list

  def box_to_dict(self,box):
	    b={"x1":box[1],"x2":box[3],"y1":box[0],"y2":box[2]}
	    return b
  
  def upscale_bbox(self,bbox):
    x1=bbox["x1"]*2
    x2=bbox["x2"]*2
    y1=bbox["y1"]*2
    y2=bbox["y2"]*2
    b={"x1":x1,"x2":x2,"y1":y1,"y2":y2}
    return b
  
  def upscale_all_bboxes(self,bbox_list):
    upscaled_bboxes=list()
    for i in range(len(bbox_list)):
      boxes=list()
      for j in bbox_list[i]:
        boxes.append(self.upscale_bbox(j))
      upscaled_bboxes.append(boxes)
      del boxes
    return upscaled_bboxes

  def upscale_bboxes_list(self,bbox_list):
    upscaled_bboxes=list()
    for i in bbox_list:
      upscaled_bboxes.append(self.upscale_bbox(i))
    return upscaled_bboxes


  # only for debugging
  def drawBox(self,image, bbox):
    im = cv2.imread(image)
    for b in bbox:
        y1 = b["y1"]
        x1 = b["x1"]
        y2 = b["y2"]
        x2 = b["x2"]

        im[y1:y2, x1:x1 + 5] = (0, 0, 0)
        im[y1:y2, x2:x2 + 5] = (0, 0, 0)
        im[y1:y1 + 5, x1:x2] = (0, 0, 0)
        im[y2:y2 + 5, x1:x2] = (0, 0, 0)
    im = cv2.resize(im, None, fx=0.3, fy=0.3)
    cv2.imshow('title', im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


  

Cropping Class

In [0]:
# Cutting of frames according to the rider
class Cropping:
    def __init__(self):
      pass
 
      # crop one frame
    def complete_bbox(self, frame,bboxes):
        sizeX = 1280  # HD
        sizeY = 720
        size = (sizeX, sizeY)
        ratio = sizeX / sizeY

      
        box=self.filter_boxes(frame,bboxes)

        box=self.calculate_box(box,sizeX,sizeY,frame)
    
        return box
   
    # create big bounding box by using max and min values for edges
    def filter_boxes(self, frame,box_list):

      # collect all x, y values 
      X1arr = []
      Y1arr = []
      X2arr = []
      Y2arr = []
              
      for j in box_list:
        X1arr.append(j["x1"])
        Y1arr.append(j["y1"])
        X2arr.append(j["x2"])
        Y2arr.append(j["y2"])

      # case no boxes are found    
      if(len(X1arr)==0):
        calculated_box={"x1":-1,"x2":-1,"y1":-1,"y2":-1,"frame":frame}
      else:
        X1 = min(X1arr)
        Y1 = min(Y1arr)
        X2 = max(X2arr)
        Y2 = max(Y2arr)
        calculated_box={"x1":X1,"x2":X2,"y1":Y1,"y2":Y2,"frame":frame}
      
      return calculated_box

    # consider borders and ratio of frames for cropping
    def calculate_box(self,box,sizeX,sizeY,img):
      X1 = box["x1"]
      Y1 = box["y1"]
      X2 = box["x2"]
      Y2 = box["y2"]

      TotalYPixels = Y2 - Y1  # Total amount of pixels of the bounding box in Y direction
      TotalXPixels=X2-X1
      TotalYPixels=1.2*TotalYPixels
      TotalXPixels=1.2*TotalXPixels

      if(TotalXPixels>TotalYPixels):
        if TotalXPixels < 480: TotalXPixels = 480
        XRatio=sizeX/TotalXPixels
        YRatio=sizeY/XRatio
        TotalYPixels = YRatio  # Total amount of pixel of the bounding box in X direction
      else:
        if TotalYPixels < 480: TotalYPixels = 480
        YRatio=sizeY/TotalYPixels
        XRatio=sizeX/YRatio
        TotalXPixels = XRatio  # Total amount of pixel of the bounding box in X direction
                
      # Calculate optimal Crop points to maintain ratio
      CropX1 = ((X2 + X1) / 2) - (TotalXPixels / 2)  # Defines the left most pos to crop the pic
      CropX2 = ((X2 + X1) / 2) + (TotalXPixels / 2)  # Defines the right most pos to crop the pic
      CropY1 = ((Y2 + Y1) / 2) - (TotalYPixels / 2)  # Defines the top most pos to crop the pic
      CropY2 = ((Y2 + Y1) / 2) + (TotalYPixels / 2)  # Defines the bottom most pos to crop the pic
                
      # Out of Bounds check
      if CropX1 < 0:  # If we are out of bounds to the left side, adjust both X pos to the right
        CropX2 += -1 * CropX1
        CropX1 = 0

      if CropX2 > img.shape[1]:  # If we are out of bounds to the right side, adjust both X pos to the left
        CropX1 += (img.shape[1] - CropX2)
        CropX2 = img.shape[1]

      if CropY1 < 0:  # If we are out of bounds to the upper side, adjust both Y pos down
        CropY2 += -1 * CropY1
        CropY1 = 0

      if CropY2 > img.shape[0]:  # If we are out of bounds to the lower side, adjust both Y pos up
        CropY1 += (img.shape[0] - CropY2)
        CropY2 = img.shape[0]

                
                
      CropX1=int(CropX1)
      CropX2=int(CropX2)
      CropY1=int(CropY1)
      CropY2=int(CropY2)
      box=self.make_bbox( CropX1,CropX2,CropY1,CropY2)
      return box

    # crop picture by x,y values
    def crop_image(self,box,img):
      X1 = box["x1"]
      Y1 = box["y1"]
      X2 = box["x2"]
      Y2 = box["y2"]
      if X1 != img.shape[1] and Y1 != img.shape[0]:
          img = img[Y1:Y2,X1:X2]
      return img
    
    def make_bbox(self, x1,x2,y1,y2):
      b={"x1":x1,"x2":x2,"y1":y1,"y2":y2}
      return b

                


Video quality class

In [0]:
import queue
class VideoQuality:
  # get last bbox if none found in current frame
  def fill_missing_bboxes(self,bboxes):
    for i in range(len(bboxes)):
      if(len(bboxes[i])==0 and i>0):
        bboxes[i]=bboxes[i-1]
    return bboxes

  # calculate bbox by last 2 frames
  def calculate_missing_bbox_last_frames(self,bboxes):
    if bboxes.empty():
      return self.make_bbox(-1,-1,-1,-1)
    if bboxes.qsize()==1:
      return bboxes.get()
    while bboxes.qsize()>2:
      bboxes.get()
    # bboxes from last 2 frames
    second=bboxes.get()
    first=bboxes.get()

    box=self.mean_boxes(first,second)
    return box

  def mean_boxes(self, first, second):
    x1=int((first["x1"]+second["x1"])/2)
    x2=int((first["x2"]+second["x2"])/2)
    y1=int((first["y1"]+second["y1"])/2)
    y2=int((first["y2"]+second["y2"])/2)
    return make_bbox(x1,x2,y1,y2)

  def make_bbox(self, x1,x2,y1,y2):
      b={"x1":x1,"x2":x2,"y1":y1,"y2":y2}
      return b

Video Class

In [0]:
import queue

class VideoProcessing:

    def __init__(self,path_weights):
      self.detector=rider_detection(path_weights)
      self.cropper=Cropping()
      self.vidQuality=VideoQuality()

    def testVideo(self,path):
      
      cap = cv2.VideoCapture(path)
      fps = cap.get(cv2.CAP_PROP_FPS)

      sizeX = 640  
      sizeY = 480
      size = (sizeX, sizeY)
      ratio = sizeX / sizeY

      # Define the codec and create VideoWriter object
      fourcc = cv2.VideoWriter_fourcc(*"mp4v")
      out = cv2.VideoWriter('/content/drive/My Drive/CV Praktikum/Test.mp4',fourcc, fps, size)

      while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
         
          frame=cv2.resize(frame,size)

          # write the flipped frame
          out.write(frame)
        else:
          break

    	# Release everything if job is finished
      cap.release()
      out.release()
    
    def process_frames(self, video_path: str):
        video = cv2.VideoCapture(video_path)
        filename = os.path.basename(video_path)
        total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)

        print(total_frames)

        fps = video.get(cv2.CAP_PROP_FPS)

        # Check for first picture
        success, img = video.read()

        sizeX = 800  # HD
        sizeY = 450
        size = (sizeX, sizeY)
        ratio = sizeX / sizeY
         
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter('/content/drive/My Drive/CV Praktikum/Cropping.mp4', fourcc, fps, size)



        
        last_frames=queue.Queue(10)
        last_bboxes=queue.Queue(10)
        for i in tnrange(int(total_frames),desc='extract frames'):
            
            if success:
              # save last 10 images in queue
              if last_frames.qsize()==10:
                last_frames.get()
              last_frames.put(img)

              #downsize frame and detect
              #downsize_img=self.downscale_frame(img,0.5)
              bboxes=self.detector.getBboxes(img)

              # calculate missing bboxes and scale them up 
              #bboxes=self.detector.upscale_bboxes_list(bboxes)

              # crop frame
              box=self.cropper.complete_bbox(img,bboxes)
              
              if(len(bboxes)==0):
                box=self.vidQuality.calculate_missing_bbox_last_frames(last_bboxes)
              
                
              cropped_frame=self.cropper.crop_image(box,img)

              # save calculated box for frame
              if last_bboxes.qsize()==10:
                last_bboxes.get()
              last_bboxes.put(box)

              # resize picture to output size
              img = cv2.resize(cropped_frame, size)
              out.write(img)
            del img
            success, img = video.read()
        video.release();
        out.release()
        

    def downscale_frames(self,frames):
      small_frames=list()
      for i in tnrange(len(frames),desc='downsize frames'):
        small_frames.append(cv2.resize(frames[i], (0,0), fx=0.5, fy=0.5) )
      return small_frames

    def downscale_frame(self,frame,scale):
      return cv2.resize(frame, (0,0), fx=scale, fy=scale)
    
    

        #videoclip = mpe.VideoFileClip(filename.strip('.MP4') + 'Crop.mp4')
        # Audiostuff, takes alot longer tho
        # audioclip = mpe.CompositeAudioClip([mpe.AudioFileClip(video_path)])
        # videoclip.audio = audioclip
        # videoclip.write_videofile(filename.strip('.mp4')+'Crop.mp4')

In [14]:
v=VideoProcessing('/content/drive/My Drive/CV Praktikum/mask_rcnn_rider_cfg_0006.h5')
v.process_frames('/content/drive/My Drive/CV Praktikum/ZOOM0004_1.mp4')

527.0


Operator Class

Main: Detection test

Verbesserungen:


*   nicht in jedem frame detektieren
*   videoqualität verbessern (smoother)
*   bei fehlenden bboxen mittelwerte nuten
*   beim filtern nur ein Pferd detektieren
*   mehr Reiter/Pferde trainieren
*   Test mit Publikum
*   Reiterpaar detektieren
*  lineare regression für fehlende bboxen
*  Spiegel
*  Kalmanfilter
*  Visualisierung von BBoxen

Performance:


*   resize (teilweise integriert)
*   weniger frames detektieren
*   frame Listen nicht zwischenspeichern/ kleinere Teile damit nicht so viel Ram benötigt wird


Idee:
evtl. die frames nicht zwischenspeichern sondern einzeln bearbeiten, dabei die letzen 10 in einer queue zwischenspeichern für Berechnungen




